# Stundenplanungsproblem - MIP Modell

Cplex wird für die Modellierung verwendet (Falls Sie sich für Python entscheiden, ziehen Sie allenfalls Pyomo oder PuLP in Betracht: diese Modellierungs-Libraries sind unabhängig von einem bestimmten Solver).

Für einen Prototypen können Sie grundsätzlich auch Julia (JuMP) oder eine Modellierungssprache (AMPL etc.) verwenden.

In [1]:
from docplex.mp.model import Model

Wir verwenden Pandas um die Daten einzulesen:

In [2]:
import pandas as pd

Daten Einlesen:

In [3]:
df = pd.read_csv('data.csv')

In [4]:
df.head()

,ID,Fach,Lehrperson,Stunden,Klasse,Overlap
0,1,SH,bip,2,U21a,23
1,2,NT,arj,2,U21a,0
2,3,RE,blb,2,U21a,0
3,4,DE,scn,2,U21a,0
4,5,TG,gej,2,U21a,0


Die Lektionen bringe ich in eine andere Struktur um später einfacher damit arbeiten zu können:

In [5]:
lectures = list()
classes = set()
teachers = set()
subjects = set()
lectures_by_class = dict()
lectures_by_teacher = dict()
lecture_details = dict()

for index, row in df.iterrows():
    lectures.append(row['ID'])
    lecture_details[row['ID']] = {'subject': row['Fach'], 'teacher': row['Lehrperson'], 'hours': row['Stunden'], 'class': row['Klasse'], 'overlap': row['Overlap']}
    classes.add(row['Klasse'])
    teachers.add(row['Lehrperson'])
    subjects.add(row['Fach'])
    if row['Klasse'] in lectures_by_class:
        lectures_by_class[row['Klasse']].append(row['ID'])
    else:
        lectures_by_class[row['Klasse']] = [row['ID']]
    if row['Lehrperson'] in lectures_by_teacher:
        lectures_by_teacher[row['Lehrperson']].append(row['ID'])
    else:
        lectures_by_teacher[row['Lehrperson']] = [row['ID']]

In [6]:
lectures

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44]

In [7]:
lecture_details

{1: {'subject': 'SH',
  'teacher': 'bip',
  'hours': 2,
  'class': 'U21a',
  'overlap': 23},
 2: {'subject': 'NT',
  'teacher': 'arj',
  'hours': 2,
  'class': 'U21a',
  'overlap': 0},
 3: {'subject': 'RE',
  'teacher': 'blb',
  'hours': 2,
  'class': 'U21a',
  'overlap': 0},
 4: {'subject': 'DE',
  'teacher': 'scn',
  'hours': 2,
  'class': 'U21a',
  'overlap': 0},
 5: {'subject': 'TG',
  'teacher': 'gej',
  'hours': 2,
  'class': 'U21a',
  'overlap': 0},
 6: {'subject': 'FR',
  'teacher': 'ebm',
  'hours': 2,
  'class': 'U21a',
  'overlap': 0},
 7: {'subject': 'EN',
  'teacher': 'dea',
  'hours': 2,
  'class': 'U21a',
  'overlap': 0},
 8: {'subject': 'MU',
  'teacher': 'vom',
  'hours': 2,
  'class': 'U21a',
  'overlap': 0},
 9: {'subject': 'DE',
  'teacher': 'scn',
  'hours': 2,
  'class': 'U21a',
  'overlap': 0},
 10: {'subject': 'KS',
  'teacher': 'bip',
  'hours': 1,
  'class': 'U21a',
  'overlap': 0},
 11: {'subject': 'BL',
  'teacher': 'bip',
  'hours': 1,
  'class': 'U21a',
  

In [8]:
lectures_by_teacher

{'bip': [1, 10, 11, 16, 18, 23, 38],
 'arj': [2, 41],
 'blb': [3, 39],
 'scn': [4, 9, 14],
 'gej': [5, 22],
 'ebm': [6, 21],
 'dea': [7, 20],
 'vom': [8],
 'ale': [12, 17, 19],
 'frc': [13, 28],
 'dua': [15, 26],
 'buk': [24, 32, 34, 40],
 'gyg': [25, 33],
 'frm': [27, 29, 30, 35, 43],
 'fsl': [31, 44],
 'jeg': [36, 42],
 'hlu': [37]}

Zusätzlich definieren wir unsere Tage und Zeitslots:

In [9]:
days = [0,1,2,3,4] # Mo, Di, Mi...
slots = [0,1,2,3,4,5,6,7,8,9,10] # 08:00 - 08:45, 08:50 - 09:35, 09:55 - 10:40...

Damit haben wir alle Daten und können unser Modell und unsere Variablen definieren:

In [10]:
model = Model('Stundenplanungsproblem HSLU')

x = {(l,d,s): model.binary_var() for l in lectures for d in days for s in slots} # Lektion l startet an Tag d in Zeitslot s
xx = {(l,d,s): model.binary_var() for l in lectures for d in days for s in slots} # Hilfsvariable

Nun kann ich sämtliche Bedingungen als Constraints definieren:

In [11]:
# Jede Lektion muss zwingend genau einmal eingeplant werden:

for l in lectures:
    model.add(model.sum(x[(l,d,s)] for d in days for s in slots) == 1)
    # Die Hilfsvariablen funktionieren ähnlich wie die Hauptvariablen, jedoch bringen Sie zum Ausdruck dass in einem bestimmten Zeitslot eine bestimmte Lektion nicht nur startet, sondern stattfindet. 
    # Dies ist relevant für Lektionen die über mehr als eine Stunde stattfinden:
    model.add(model.sum(xx[(l,d,s)] for d in days for s in slots) == lecture_details[l]['hours'])
    for d in days:
        for s in slots:
            model.add(x[(l,d,s)] <= xx[(l,d,s)]) # Wenn Lektion l in einem Zeitslot (d,s) startet, findet sie in diesem Zeitslot auch statt
            if lecture_details[l]['hours'] == 2:
                if s < len(slots)-1:
                    model.add(x[(l,d,s)] <= xx[(l,d,s+1)]) # Wenn eine zweistündige Lektion in einem Zeitslot (d,s) startet, findet sie im Zeitslot (d,s+1) noch immer statt
                else:
                    model.add(x[(l,d,s)] == 0) # Eine zweistündige Lektion kann nicht im letzten Zeitslot eines Tages eingeplant werden

In [12]:
# Aus der Sicht der Klassen darf es keine Überlappungen von Stunden geben:

for c in classes:
    for d in days:
        for s in slots:
            model.add(model.sum(xx[(l,d,s)] for l in lectures_by_class[c]) <= 1)

In [13]:
# Aus der Sicht der Lehrpersonen darf es ebenfalls keine Überlappungen von Lektionen geben
# Einzige Ausnahme sind Lektionen, für die ein Overlap zugelassen sind (z.B. Sport wird von einer Lehrperson für beide Klassen gleichzeitig unterrichtet)

for t in teachers:
    for d in days:
        for s in slots:
            model.add(model.sum(xx[(l,d,s)] for l in lectures_by_teacher[t] if lecture_details[l] == 0) <= 1) # Ich habe es mir hier etwas einfach gemacht, eigentlich reicht diese Bedingung nicht, funktioniert hier aber

In [14]:
# Gewisse Lektionen finden gleichzeitig statt (z.B. die Sportlektionen der beiden Klassen)

for l in lectures:
    overlap = lecture_details[l]['overlap']
    if overlap > 0:
        for d in days:
            for s in slots:
                model.add(x[(l,d,s)] == x[overlap,d,s])

Damit haben wir die Grundstruktur und die wichtigsten Regeln festgelegt und können nun grundsätzlich einen Stundenplan generieren, indem wir den Solver (hier: Cplex) aufrufen:

In [15]:
model.solve(log_output = True)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 2 times.
MIP Presolve eliminated 2592 rows and 140 columns.
MIP Presolve modified 1260 coefficients.
Aggregator did 105 substitutions.
Reduced MIP has 2616 rows, 4595 columns, and 13115 nonzeros.
Reduced MIP has 4595 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (14.96 ticks)
Found incumbent of value 0.000000 after 0.04 sec. (32.56 ticks)

Root node processing (before b&c):
  Real time             =    0.05 sec. (32.75 ticks)
Parallel b&c, 12 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.05 sec. (32.75 ticks)


docplex.mp.solution.SolveSolution(obj=0,values={x1:1,x79:1,x118:1,x217:1..

Zur Visualisierung des Resultates erstelle ich kurz eine Funktion, die ich später wiederverwenden kann:

In [16]:
def visualize():
    for c in classes:
        df = pd.DataFrame(columns=days, index=slots)
        for d in days:
            for s in slots:
                empty = True
                for l in lectures_by_class[c]:
                    if xx[(l,d,s)].solution_value > 0.5:
                        df.at[s,d] = lecture_details[l]['subject'] + ' / ' + lecture_details[l]['teacher']
                        empty = False
                if empty:
                    df.at[s,d] = ''
        from IPython.display import display, HTML
        print(c)
        display(HTML(df.to_html()))

In [17]:
visualize()

U21a


,0,1,2,3,4
0,SH / bip,MA / ale,,BG / gej,
1,SH / bip,KS / bip,NT / arj,BG / gej,MU / vom
2,,TG / gej,NT / arj,,MU / vom
3,BL / bip,TG / gej,,,GG / bip
4,DE / scn,,GS / dua,,GG / bip
5,DE / scn,,GS / dua,EN / dea,FR / ebm
6,EN / dea,MA / ale,FR / ebm,EN / dea,
7,RE / blb,MA / ale,FR / ebm,,DE / scn
8,RE / blb,,,IN / frc,DE / scn
9,DE / scn,,MA / ale,SH / bip,


U21b


,0,1,2,3,4
0,SH / bip,,EN / fsl,GS / dua,
1,SH / bip,FR / jeg,EN / fsl,GS / dua,BL / frm
2,,DE / frm,,NT / arj,KS / frm
3,GG / buk,DE / frm,DE / frm,NT / arj,
4,GG / buk,IN / frc,DE / frm,,
5,MA / buk,DE / frm,,,
6,,,,MU / hlu,
7,FR / jeg,,BG / gyg,MU / hlu,TG / gyg
8,FR / jeg,,BG / gyg,,TG / gyg
9,MA / buk,RE / blb,EN / fsl,SH / bip,MA / buk


Damit haben wir nun grundsätzlich einen Stundenplan. Dieser hat aber noch viele Mängel.

Beispielsweise:

- Keine geregelten Mittagspausen
- Mittwochnachmittag ist nicht frei
- Schlechte Stundenverteilung
- Tage an denen das selbe Fach zwei Mal unterrichtet wird
- etc.

Fügen wir noch einige Bedingungen hinzu:

In [18]:
# Der Mittwochnachmittag soll für beide Klassen frei bleiben:

for c in classes:
    for s in slots:
        if s >= 5:
            for l in lectures:
                model.add(x[(l,2,s)] == 0)

In [19]:
model.solve()

docplex.mp.solution.SolveSolution(obj=0,values={x5:1,x76:1,x134:1,x216:1..

In [20]:
visualize()

U21a


,0,1,2,3,4
0,MU / vom,,MA / ale,,GS / dua
1,MU / vom,,RE / blb,GG / bip,GS / dua
2,BG / gej,,RE / blb,GG / bip,
3,BG / gej,,,DE / scn,
4,SH / bip,FR / ebm,IN / frc,,
5,SH / bip,EN / dea,,DE / scn,
6,MA / ale,EN / dea,,DE / scn,DE / scn
7,MA / ale,,,TG / gej,DE / scn
8,MA / ale,BL / bip,,TG / gej,KS / bip
9,MA / ale,NT / arj,,FR / ebm,EN / dea


U21b


,0,1,2,3,4
0,,,DE / frm,DE / frm,
1,MA / buk,MA / buk,,DE / frm,GS / dua
2,MA / buk,MA / buk,EN / fsl,MU / hlu,GS / dua
3,,KS / frm,,MU / hlu,
4,SH / bip,MA / buk,FR / jeg,BL / frm,
5,SH / bip,DE / frm,FR / jeg,IN / frc,EN / fsl
6,,DE / frm,,GG / buk,EN / fsl
7,NT / arj,,,GG / buk,
8,NT / arj,,,RE / blb,BG / gyg
9,FR / jeg,TG / gyg,,RE / blb,BG / gyg


In [21]:
# Slot 5 soll grundsätzlich als Mittagspause frei bleiben:

model.add(model.sum(xx[(l,d,5)] for l in lectures for d in days) == 0)
# Bemerkung: wie sie sehen habe ich diese Bedingung kompakter formuliert als die vorherige. Es gibt immer verschiedene alternative Formulierungen, diese sind jedoch teilweise unterschiedlich effizient
model.solve()
visualize()

U21a


,0,1,2,3,4
0,NT / arj,MA / ale,BL / bip,FR / ebm,KS / bip
1,NT / arj,BG / gej,RE / blb,FR / ebm,DE / scn
2,,BG / gej,RE / blb,MA / ale,DE / scn
3,GG / bip,DE / scn,TG / gej,MA / ale,
4,GG / bip,DE / scn,TG / gej,,IN / frc
5,,,,,
6,SH / bip,MU / vom,,,DE / scn
7,SH / bip,MU / vom,,MA / ale,
8,,EN / dea,,MA / ale,
9,SH / bip,GS / dua,,EN / dea,FR / ebm


U21b


,0,1,2,3,4
0,RE / blb,,TG / gyg,MA / buk,FR / jeg
1,RE / blb,GG / buk,TG / gyg,MA / buk,
2,MA / buk,GG / buk,DE / frm,,
3,MU / hlu,,MA / buk,FR / jeg,KS / frm
4,MU / hlu,IN / frc,MA / buk,FR / jeg,
5,,,,,
6,SH / bip,,,,DE / frm
7,SH / bip,NT / arj,,BG / gyg,DE / frm
8,,NT / arj,,BG / gyg,GS / dua
9,SH / bip,EN / fsl,,DE / frm,GS / dua


In [22]:
# Die letzte Stunde (10) soll nicht verwendet werden. Zudem soll auch Lektion 9 so wenig wie möglich verwendet werden

model.add(model.sum(x[(l,d,10)] for l in lectures for d in days) == 0)
slot_9_used = 0
for d in days:
    slot_9_used += model.sum(xx[(l,d,9)] for l in lectures for d in days)
    
model.minimize(slot_9_used) # Zielfunktion
# Wichtig: stellen Sie sicher, dass sie verstehen, dass Slot 10 auf keinen Fall verwendet werden darf, Slot 9 grundsätzlich schon. 
# Slot 10 wird explizit verboten, Slot 9 wird nur "unattraktiv gemacht" indem wir seine Verwendung in der Zielfunktion bestrafen
model.solve(log_output=True)
visualize()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Retaining values of one MIP start for possible repair.
Tried aggregator 2 times.
MIP Presolve eliminated 3664 rows and 976 columns.
MIP Presolve modified 756 coefficients.
Aggregator did 77 substitutions.
Reduced MIP has 2102 rows, 3787 columns, and 10575 nonzeros.
Reduced MIP has 3787 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (12.78 ticks)
Found incumbent of value 40.000000 after 0.04 sec. (28.64 ticks)
Probing fixed 144 vars, tightened 0 bounds.
Probing changed sense of 656 constraints.
Probing time = 0.02 sec. (7.64 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 672 rows and 800 columns.
Reduced MIP has 1430 rows, 2987 columns, and 8319 nonzeros.
Reduced MIP has 2987 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (8.17 ticks)
Probing time = 0.01 sec. (3.37

,0,1,2,3,4
0,EN / dea,SH / bip,GG / bip,MA / ale,FR / ebm
1,EN / dea,MA / ale,GG / bip,MA / ale,FR / ebm
2,TG / gej,MA / ale,EN / dea,BG / gej,KS / bip
3,TG / gej,SH / bip,RE / blb,BG / gej,GS / dua
4,DE / scn,SH / bip,RE / blb,IN / frc,GS / dua
5,,,,,
6,MU / vom,BL / bip,,FR / ebm,DE / scn
7,MU / vom,DE / scn,,NT / arj,DE / scn
8,,DE / scn,,NT / arj,MA / ale
9,,,,,


U21b


,0,1,2,3,4
0,DE / frm,SH / bip,BL / frm,BG / gyg,GG / buk
1,DE / frm,NT / arj,EN / fsl,BG / gyg,GG / buk
2,DE / frm,NT / arj,EN / fsl,FR / jeg,GS / dua
3,TG / gyg,SH / bip,DE / frm,MU / hlu,GS / dua
4,TG / gyg,SH / bip,DE / frm,MU / hlu,KS / frm
5,,,,,
6,MA / buk,EN / fsl,,MA / buk,RE / blb
7,MA / buk,FR / jeg,,MA / buk,RE / blb
8,IN / frc,FR / jeg,,,MA / buk
9,,,,,


In [23]:
# Das selbe Fach soll nicht mehrfach am selben Tag in der selben Klasse unterrichtet werden

for c in classes:
    for d in days:
        for su in subjects:
            model.add(model.sum(x[(l,d,s)] for s in slots for l in lectures_by_class[c] if lecture_details[l]['subject'] == su) <= 1)
            
model.solve()
visualize()    

U21a


,0,1,2,3,4
0,BG / gej,EN / dea,GG / bip,DE / scn,DE / scn
1,BG / gej,NT / arj,GG / bip,DE / scn,DE / scn
2,MA / ale,NT / arj,SH / bip,RE / blb,FR / ebm
3,MA / ale,MU / vom,TG / gej,RE / blb,FR / ebm
4,FR / ebm,MU / vom,TG / gej,,KS / bip
5,,,,,
6,DE / scn,MA / ale,,IN / frc,MA / ale
7,GS / dua,SH / bip,,EN / dea,MA / ale
8,GS / dua,SH / bip,,EN / dea,BL / bip
9,,,,,


U21b


,0,1,2,3,4
0,DE / frm,RE / blb,MA / buk,FR / jeg,DE / frm
1,DE / frm,RE / blb,MA / buk,FR / jeg,GS / dua
2,BG / gyg,NT / arj,SH / bip,IN / frc,GS / dua
3,BG / gyg,NT / arj,TG / gyg,GG / buk,MA / buk
4,EN / fsl,MA / buk,TG / gyg,GG / buk,MA / buk
5,,,,,
6,,FR / jeg,,DE / frm,EN / fsl
7,MU / hlu,SH / bip,,DE / frm,EN / fsl
8,MU / hlu,SH / bip,,BL / frm,KS / frm
9,,,,,


In [24]:
# Gewisse Fächer, z.B. Sport, sollen innerhalb von 3 Tagen maximal einmal unterrichtet werden, damit die Regeneration gewährleistet ist

for c in classes:
    for d in days[:-2]:
        model.add(model.sum(x[(l,dd,s)] for dd in [d,d+1,d+2] for s in slots for l in lectures_by_class[c] if lecture_details[l]['subject'] == 'SH') <= 1)
            
model.solve()
visualize()    

U21a


,0,1,2,3,4
0,EN / dea,DE / scn,GG / bip,DE / scn,TG / gej
1,EN / dea,DE / scn,GG / bip,DE / scn,TG / gej
2,SH / bip,IN / frc,GS / dua,KS / bip,RE / blb
3,MU / vom,MA / ale,GS / dua,BG / gej,RE / blb
4,MU / vom,MA / ale,,BG / gej,EN / dea
5,,,,,
6,BL / bip,NT / arj,,MA / ale,MA / ale
7,FR / ebm,NT / arj,,SH / bip,MA / ale
8,FR / ebm,FR / ebm,,SH / bip,DE / scn
9,,,,,


U21b


,0,1,2,3,4
0,DE / frm,IN / frc,MA / buk,DE / frm,RE / blb
1,DE / frm,BG / gyg,MA / buk,DE / frm,RE / blb
2,SH / bip,BG / gyg,MU / hlu,KS / frm,
3,MA / buk,FR / jeg,MU / hlu,GS / dua,EN / fsl
4,MA / buk,FR / jeg,BL / frm,GS / dua,EN / fsl
5,,,,,
6,TG / gyg,MA / buk,,FR / jeg,DE / frm
7,TG / gyg,NT / arj,,SH / bip,GG / buk
8,EN / fsl,NT / arj,,SH / bip,GG / buk
9,,,,,


In [25]:
# Lehrer "bip" möchte maximal 2 Tage pro Woche arbeiten
# hierzu führen wir für jeden Tag eine Hilfsvariable ein:

bip_arbeitet = {d: model.binary_var() for d in days}

# Nun können wir diese Variable mit den x Variablen verknüpfen:
for d in days:
    model.add(model.sum(x[(l,d,s)] for l in lectures for s in slots if lecture_details[l]['teacher'] == 'bip') <= 10*bip_arbeitet[d])

model.add(model.sum(bip_arbeitet.values()) <= 2)

solution = model.solve(log_output = True)
visualize()
for d in days:
    if bip_arbeitet[d].solution_value > 0.5:
        print('bip arbeitet an Tag %i'%d)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Retaining values of one MIP start for possible repair.
Tried aggregator 2 times.
MIP Presolve eliminated 3797 rows and 985 columns.
MIP Presolve modified 783 coefficients.
Aggregator did 77 substitutions.
Reduced MIP has 2131 rows, 3783 columns, and 11578 nonzeros.
Reduced MIP has 3783 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (16.17 ticks)
Probing fixed 154 vars, tightened 0 bounds.
Probing changed sense of 646 constraints.
Probing time = 0.01 sec. (7.88 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 662 rows and 800 columns.
MIP Presolve modified 1 coefficients.
Reduced MIP has 1469 rows, 2983 columns, and 9332 nonzeros.
Reduced MIP has 2983 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (8.98 ticks)
Probing time = 0.00 sec. (3.40 ticks)
Tried aggregator 1 

,0,1,2,3,4
0,FR / ebm,MA / ale,MA / ale,SH / bip,DE / scn
1,NT / arj,MA / ale,GS / dua,SH / bip,DE / scn
2,NT / arj,EN / dea,GS / dua,BL / bip,FR / ebm
3,DE / scn,EN / dea,TG / gej,GG / bip,FR / ebm
4,DE / scn,DE / scn,TG / gej,GG / bip,EN / dea
5,,,,,
6,BG / gej,,,MU / vom,MA / ale
7,BG / gej,RE / blb,,MU / vom,MA / ale
8,SH / bip,RE / blb,,KS / bip,IN / frc
9,,,,,


U21b


,0,1,2,3,4
0,DE / frm,RE / blb,FR / jeg,SH / bip,BG / gyg
1,DE / frm,RE / blb,FR / jeg,SH / bip,BG / gyg
2,NT / arj,BL / frm,TG / gyg,DE / frm,MU / hlu
3,NT / arj,MA / buk,TG / gyg,DE / frm,MU / hlu
4,FR / jeg,MA / buk,MA / buk,KS / frm,IN / frc
5,,,,,
6,GG / buk,EN / fsl,,EN / fsl,MA / buk
7,GG / buk,EN / fsl,,GS / dua,MA / buk
8,SH / bip,,,GS / dua,DE / frm
9,,,,,


bip arbeitet an Tag 0
bip arbeitet an Tag 3
